# Day 22: Wizard Simulator 20XX

https://adventofcode.com/2015/day/22

In [79]:
class Spell():
    def __init__(self,name,cost,effect,damage,armor,heal,mana):
        self.name   = name
        self.cost   = cost
        self.effect = effect
        self.damage = damage
        self.armor  = armor
        self.heal   = heal
        self.mana   = mana
            
    def cast(self):
        if self.effect>0 or self.name=="M" or self.name=="D":
            self.effect -=1
            return self.damage,self.armor,self.heal,self.mana 
        else:
            return 0,0,0,0

spellList = ("M","D","S","P","R")

def CastSpell(name):
    '''Returns a fresh instance of a Spell'''
    if name=="M":
        return Spell("M",  53, 0, 4, 0, 0, 0)
    elif name=="D":
        return Spell("D",  73, 0, 2, 0, 2, 0)
    elif name=="S":
        return Spell("S", 113, 6, 0, 7, 0, 0)
    elif name=="P":
        return Spell("P", 173, 6, 3, 0, 0, 0)
    elif name=="R":
        return Spell("R", 229, 5, 0, 0, 0, 101)

S = CastSpell("R")

for i in range(1,7):
    damage, armor, heal, mana = S.cast()
    print(mana)
    
S = CastSpell("R")
damage, armor, heal, mana = S.cast()
print(mana)

101
101
101
101
101
0
101


In [80]:
class Wizard():
    def __init__(self,hp,armor,mana):
        self.hp = hp
        self.armor = armor
        self.mana = mana
    def receiveDamage(self,damage):
        self.hp -= max(1,damage-self.armor)

class Boss():
    def __init__(self,hp,damage):
        self.hp = hp
        self.damage = damage

In [149]:
from collections import defaultdict
from random import choice

def Match(p,b,diff,spellSequence,spentManaMin,verbose):
    
    i = 0
    castseq = []
    spentMana = 0
    spellReg = { "S": None , "P": None , "R": None } # only need to keep track of effect spells
    
    while True:    

        if verbose:
            print("\n--- Player turn ---")
            print("- P: HP", p.hp,"mana", p.mana)
            print("- B: HP", b.hp)

        if diff=="H": # hard
            p.hp -= 1
            if verbose: print("- P: HP", p.hp,"(Hard mode)")
            if p.hp <=0:
                if verbose: print("Player dies!")
                return -1, spentMana, castseq
                break
            
        p.armor = 0       
        if verbose: print("Using spell effects")
        for s in spellReg.keys():  
            if spellReg[s] != None:
                spell = spellReg[s]
                if spell.effect>0:
                    if verbose: print("- spell",spell.name,"effect = ",spell.effect)
                    d, a, h, m = spell.cast()
                    b.hp -= d
                    if b.hp <=0:
                        if verbose: print("Boss dies!")
                        return 1, spentMana, castseq
                        break
                    p.armor += a
                    p.hp += h
                    p.mana += m
                    if verbose: print("- spell",spell.name,"has now effect = ",spell.effect)
                   
        # pruning expired spells before trying to cast a new one
        for s in spellReg.keys():
            if spellReg[s] != None:
                if spellReg[s].effect==0:
                    spellReg[s] == None
            
        if verbose: print("Trying to cast a new spell")
        castNewSpell = True
        n = ""
        
        if i<len(spellSequence):
            # spell from external sequence, for testing purposes
            n = spellSequence[i]
            if CastSpell(n).cost>p.mana: # don't have enough mana to cast
                if verbose: print("Not enough mana to cast",n,".")
                return 0, spentMana, castseq
            i+=1
        else:
            # random spell
            spelltried = []
            while True:
                if len(spelltried)==5: # tried all spells, but cannot cast any
                    castNewSpell=False
                    break
                n = choice(spellList)
                if n in spelltried:
                    continue
                else:
                    spelltried.append(n)
                if CastSpell(n).cost<=p.mana: # have enough mana to cast
                    if n=="M" or n=="D" : # Immediate spells
                        break
                    else: # check whether effect spell still active 
                        if spellReg[n]==None:
                            break
                else:
                    continue
        
        if castNewSpell:

            if verbose: print("- spell",n)
            snew = CastSpell(n)
            castseq.append(n)

            p.mana -= snew.cost
            spentMana += snew.cost
            if spentMana >= spentManaMin:
                if verbose: print("Mana spent larger then previous minimum. Aborting fight.")
                return 0, spentMana, castseq

            if snew.effect==0:
                # act immediately
                if verbose: print(n,"act immediately!")
                d, a, h, m = snew.cast()
                p.hp += h
                p.armor += a
                p.mana += m
                b.hp -= d
                if b.hp <=0:
                    if verbose: print("Boss dies!")
                    return 1, spentMana, castseq
                    break
            else:
                spellReg[n] = snew

        if verbose:
            print("\n--- Boss turn ---")
            print("- P: HP", p.hp,"mana", p.mana)
            print("- B: HP", b.hp)

        p.armor = 0
        if verbose: print("Using spell effects")
        for s in spellReg.keys():  
            if spellReg[s] != None:
                spell = spellReg[s]
                if spell.effect>0:
                    if verbose: print("- spell",spell.name,"effect = ",spell.effect)
                    d, a, h, m = spell.cast()
                    b.hp -= d
                    if b.hp <=0:
                        if verbose: print("Boss dies!")
                        return 1, spentMana, castseq
                        break
                    p.armor += a
                    p.hp += h
                    p.mana += m
                    if verbose: print("- spell",spell.name,"has now effect = ",spell.effect)
                else:
                    spellReg[s] = None
      
        if verbose: print("B hit points =",b.hp)
        if verbose: print("P armor =",p.armor)
        p.receiveDamage(b.damage)
        if verbose: print("P hit points =",p.hp)
        if p.hp <=0:
            if verbose: print("Player dies!")
            return -1, spentMana, castseq
            break
            
p = Wizard(10,0,250)
b = Boss(13,8)

spellSequence = ["P","M"]

status, spentmana, castseq = Match(p,b,"E",spellSequence,spentManaMin=10000,verbose=True)

print("\n*** Player mana after fight =",p.mana)
print("*** Mana spent during fight =",spentmana)


--- Player turn ---
- P: HP 10 mana 250
- B: HP 13
Using spell effects
Trying to cast a new spell
- spell P

--- Boss turn ---
- P: HP 10 mana 77
- B: HP 13
Using spell effects
- spell P effect =  6
- spell P has now effect =  5
B hit points = 10
P armor = 0
P hit points = 2

--- Player turn ---
- P: HP 2 mana 77
- B: HP 10
Using spell effects
- spell P effect =  5
- spell P has now effect =  4
Trying to cast a new spell
- spell M
M act immediately!

--- Boss turn ---
- P: HP 2 mana 24
- B: HP 3
Using spell effects
- spell P effect =  4
Boss dies!

*** Player mana after fight = 24
*** Mana spent during fight = 226


In [150]:
p = Wizard(10,0,250)
b = Boss(14,8)

SpellSequence = ["R","S","D","P","M"]

status, spentmana, castseq = Match(p,b,"E",SpellSequence,spentManaMin=1000,verbose=True)

print("\n*** Player mana after fight =",p.mana)
print("*** Mana spent during fight =",spentmana)


--- Player turn ---
- P: HP 10 mana 250
- B: HP 14
Using spell effects
Trying to cast a new spell
- spell R

--- Boss turn ---
- P: HP 10 mana 21
- B: HP 14
Using spell effects
- spell R effect =  5
- spell R has now effect =  4
B hit points = 14
P armor = 0
P hit points = 2

--- Player turn ---
- P: HP 2 mana 122
- B: HP 14
Using spell effects
- spell R effect =  4
- spell R has now effect =  3
Trying to cast a new spell
- spell S

--- Boss turn ---
- P: HP 2 mana 110
- B: HP 14
Using spell effects
- spell S effect =  6
- spell S has now effect =  5
- spell R effect =  3
- spell R has now effect =  2
B hit points = 14
P armor = 7
P hit points = 1

--- Player turn ---
- P: HP 1 mana 211
- B: HP 14
Using spell effects
- spell S effect =  5
- spell S has now effect =  4
- spell R effect =  2
- spell R has now effect =  1
Trying to cast a new spell
- spell D
D act immediately!

--- Boss turn ---
- P: HP 3 mana 239
- B: HP 12
Using spell effects
- spell S effect =  4
- spell S has now eff

In [155]:
p = Wizard(10,0,250)
b = Boss(14,8)

status, spentmana, castseq = Match(p,b,"E",spellSequence=[],spentManaMin=10000,verbose=True)

print("\n*** Player mana after fight =",p.mana)
print("*** Mana spent during fight =",spentmana)
print(castseq)


--- Player turn ---
- P: HP 10 mana 250
- B: HP 14
Using spell effects
Trying to cast a new spell
- spell S

--- Boss turn ---
- P: HP 10 mana 137
- B: HP 14
Using spell effects
- spell S effect =  6
- spell S has now effect =  5
B hit points = 14
P armor = 7
P hit points = 9

--- Player turn ---
- P: HP 9 mana 137
- B: HP 14
Using spell effects
- spell S effect =  5
- spell S has now effect =  4
Trying to cast a new spell
- spell D
D act immediately!

--- Boss turn ---
- P: HP 11 mana 64
- B: HP 12
Using spell effects
- spell S effect =  4
- spell S has now effect =  3
B hit points = 12
P armor = 7
P hit points = 10

--- Player turn ---
- P: HP 10 mana 64
- B: HP 12
Using spell effects
- spell S effect =  3
- spell S has now effect =  2
Trying to cast a new spell
- spell M
M act immediately!

--- Boss turn ---
- P: HP 10 mana 11
- B: HP 8
Using spell effects
- spell S effect =  2
- spell S has now effect =  1
B hit points = 8
P armor = 7
P hit points = 9

--- Player turn ---
- P: HP 

## Part 1

Solution = 953

In [217]:
# You start with 50 hit points and 500 mana points
# Boss:
# Hit Points: 55
# Damage: 8

spentmin = 10000

while True:
    
    p = Wizard(50,0,500)
    b = Boss(55,8)
    
    status, spentmana, castseq = Match(p,b,diff="E",spellSequence=[],spentManaMin=spentmin,verbose=False)
    
    if status==1: # player wins
        if spentmana < spentmin:
            spentmin = spentmana
            print(spentmana,castseq)

        if spentmin == 953: # I should find a nicer way to stop the search! :-)
            break

1508 ['R', 'M', 'P', 'S', 'M', 'R', 'P', 'D', 'D', 'S', 'P', 'M']
1461 ['R', 'D', 'S', 'P', 'M', 'R', 'M', 'S', 'M', 'D', 'P', 'D', 'M']
953 ['P', 'R', 'M', 'M', 'P', 'S', 'M', 'M', 'M']


## Part 2

* 1295 too high :-(

In [218]:
# You start with 50 hit points and 500 mana points
# Boss:
# Hit Points: 55
# Damage: 8

spentmin = 10000

while True:
    
    p = Wizard(50,0,500)
    b = Boss(55,8)
    
    status, spentmana, castseq = Match(p,b,diff="H",spellSequence=[],spentManaMin=spentmin,verbose=False)
    
    if status==1: # player wins
        if spentmana < spentmin:
            spentmin = spentmana
            print(spentmana,castseq)

        if spentmin == 953: # I should find a nicer way to stop the search! :-)
            break

1501 ['S', 'R', 'D', 'P', 'M', 'R', 'S', 'P', 'M', 'D', 'S', 'M', 'M']
1402 ['P', 'R', 'S', 'D', 'P', 'R', 'S', 'M', 'P', 'D']
1382 ['P', 'R', 'S', 'D', 'P', 'R', 'M', 'S', 'P', 'M']
1362 ['P', 'R', 'M', 'S', 'P', 'R', 'M', 'S', 'P', 'M']
1295 ['S', 'R', 'P', 'M', 'M', 'R', 'S', 'P', 'M', 'M', 'M']


KeyboardInterrupt: 